# Démonstration du modèle TensorRT en temps réel

### Classe de capture et de traitement du flux vidéo

La classe suivante est hérité des Threading et prend en charge :

- L'initialisation de la caméra et du moteur TensorRT
- La capture du flux vidéo et son traitement avec le modèle, exécuté dans le programme principal du thread

**Pour réinitialiser la caméra :** sudo systemctl restart nvargus-daemon.service 

In [1]:
!sudo /home/jetson/notebook/ResetCam.sh

**Affiche les caméras disponnibles :**

In [2]:
!v4l2-ctl --list-devices

vi-output, imx219 7-0010 (platform:54080000.vi:0):
	/dev/video0

USB 2.0 Camera (usb-70090000.xusb-2.1):
	/dev/video1



**Affiche les informations sur la caméra USB**

In [3]:
!v4l2-ctl --all -d /dev/video1

Driver Info (not using libv4l2):
	Driver name   : uvcvideo
	Card type     : USB 2.0 Camera
	Bus info      : usb-70090000.xusb-2.1
	Driver version: 4.9.255
	Capabilities  : 0x84200001
		Video Capture
		Streaming
		Extended Pix Format
		Device Capabilities
	Device Caps   : 0x04200001
		Video Capture
		Streaming
		Extended Pix Format
Priority: 2
Video input : 0 (Camera 1: ok)
Format Video Capture:
	Width/Height      : 1280/960
	Pixel Format      : 'YUYV'
	Field             : None
	Bytes per Line    : 2560
	Size Image        : 2457600
	Colorspace        : sRGB
	Transfer Function : Default (maps to sRGB)
	YCbCr/HSV Encoding: Default (maps to ITU-R 601)
	Quantization      : Default (maps to Limited Range)
	Flags             : 
Crop Capability Video Capture:
	Bounds      : Left 0, Top 0, Width 1280, Height 960
	Default     : Left 0, Top 0, Width 1280, Height 960
	Pixel Aspect: 1/1
Selection: crop_default, Left 0, Top 0, Width 1280, Height 960
Selection: crop_bounds, Left 0, Top 0, Width 1280,

**Affiche les informations sur les résolutions disponnibles**

In [4]:
!v4l2-ctl --list-formats-ext -d /dev/video1

ioctl: VIDIOC_ENUM_FMT
	Index       : 0
	Type        : Video Capture
	Pixel Format: 'YUYV'
	Name        : YUYV 4:2:2
		Size: Discrete 640x480
			Interval: Discrete 0.033s (30.000 fps)
		Size: Discrete 352x288
			Interval: Discrete 0.033s (30.000 fps)
		Size: Discrete 320x240
			Interval: Discrete 0.033s (30.000 fps)
		Size: Discrete 160x120
			Interval: Discrete 0.033s (30.000 fps)
		Size: Discrete 1280x720
			Interval: Discrete 0.033s (30.000 fps)
		Size: Discrete 1280x960
			Interval: Discrete 0.033s (30.000 fps)

	Index       : 1
	Type        : Video Capture
	Pixel Format: 'MJPG' (compressed)
	Name        : Motion-JPEG
		Size: Discrete 640x480
			Interval: Discrete 0.033s (30.000 fps)
		Size: Discrete 352x288
			Interval: Discrete 0.033s (30.000 fps)
		Size: Discrete 320x240
			Interval: Discrete 0.033s (30.000 fps)
		Size: Discrete 160x120
			Interval: Discrete 0.033s (30.000 fps)
		Size: Discrete 1280x720
			Interval: Discrete 0.033s (30.000 fps)
		Size: Discrete 1280x960
			Inter

In [5]:
import tensorrt as trt
import numpy as np
import pycuda.driver as cuda
import threading
import ctypes
import time
import traitlets
import atexit
import cv2

class TRTInference(threading.Thread):
    def __init__(self,repertoire_engine, widget_image,type_camera,capture_device,capture_width,capture_height,display_width,display_height,fps,flip=0):
        threading.Thread.__init__(self)
        self.widget_image = widget_image
        self.type_camera = type_camera
        self.capture_device = capture_device
        self.capture_width = capture_width
        self.capture_height = capture_height
        self.display_width = display_width
        self.display_height = display_height
        self.fps = fps
        self.flip = flip
        self.camera_on = False

        # Initialisation des variables de la caméra
        self._running = False
        self.image_pour_widget = np.zeros((capture_width, capture_height, 3), dtype=np.int32)
        self.image = np.zeros((256, 256, 3), dtype=np.int32)
        
        if self.type_camera.find("CSI")>=0:
            self.cap = cv2.VideoCapture(self._gstreamer_pipeline_CSI(),cv2.CAP_GSTREAMER)
        else:
            self.cap = cv2.VideoCapture(self._gstreamer_pipeline_USB(),cv2.CAP_GSTREAMER)
        if self.cap.isOpened():
            print("Caméra initialisée")
        else:
            print("Erreur d'ouverture du flux vidéo")
        atexit.register(self.cap.release)

        # Initialisation du runtime TensorRT
        #self.logger = MyLogger()
        self.logger = trt.Logger(trt.Logger.INFO)
        trt.init_libnvinfer_plugins(self.logger, namespace="")
        self.runtime = trt.Runtime(self.logger)
        
        # Chargement du moteur
        print("Chargement du moteur...")
        with open(repertoire_engine, "rb") as f:
            self.engine = self.runtime.deserialize_cuda_engine(f.read())
        
        self.context = self.engine.create_execution_context()
 
        #Initialisation du context Cuda et du contexte TensorRT 
        cuda.init()
        self.cudactx = cuda.Device(0).retain_primary_context()
        self.cudactx.push()
        self.context.debug_sync = True
        
        # Réservation de la mémoire pour l'entrée
        print("Allocation mémoire...")
        size_input = trt.volume(self.engine.get_binding_shape(0))*self.engine.max_batch_size
        self.input_host_mem = cuda.pagelocked_empty(size_input, np.int32)
        self.input_device_mem = cuda.mem_alloc(self.input_host_mem.nbytes)
       
        # Réservation de la mémoire pour les sorties
        self.output_device_mem = [];
        format_sorties = [];
        types_sorties = [];

        for i in range(self.engine.num_bindings):
            if not self.engine.binding_is_input(i):
                size_output = trt.volume(self.engine.get_binding_shape(i))*self.engine.max_batch_size
                output_hm = cuda.pagelocked_empty(size_output,trt.nptype(self.engine.get_binding_dtype(i)))
                self.output_device_mem.append(cuda.mem_alloc(output_hm.nbytes))
                format_sorties.append(self.engine.get_binding_shape(i))
                types_sorties.append(trt.nptype(self.engine.get_binding_dtype(i)))

        # Récupère les adresses en GPU des buffers entrées / sorties
        binding_entree = int(self.input_device_mem)
        binding_sorties = []

        for output_ in self.output_device_mem:
            binding_sorties.append(int(output_))
        self.bindings = [binding_entree, binding_sorties[0]]

        # Allocation de la mémoire hote pour les sorties
        self.output_host_mem = []
        for i in range(len(self.output_device_mem)):
            self.output_host_mem.append(np.zeros(format_sorties[i],types_sorties[i]))
        
        # Input tensor
        self.image = np.zeros((256, 256, 3), dtype=trt.nptype(self.engine.get_binding_dtype(0)))
        self.image = self.image.astype(np.int32)       
        self.cudactx.pop()

    # Inférence
    def Calcul(self):
        # Copie de l'image dans le tenseur d'entrée
        x = self.image.astype(np.int32)
        x = np.expand_dims(x,axis=0)                    # (1,256,256,3)
        np.copyto(self.input_host_mem,x.ravel())
        
        # Transfert de l'entrée vers le GPU
        self.cudactx = cuda.Device(0).retain_primary_context()
        self.cudactx.push()
        cuda.memcpy_htod(self.input_device_mem, self.input_host_mem)
        
        # Appel du modèle
        self.context.execute_v2(bindings=self.bindings)
        
        # Récupération des sorties
        for i in range(len(self.output_host_mem)):
            cuda.memcpy_dtoh(self.output_host_mem[i], self.output_device_mem[i])
        self.cudactx.pop()

    # Lecture d'une frame
    def capture_image(self):
        re, image = self.cap.read()
        if re:
            return image
        else:
            return self.widget_image.value
        
    def run(self):
        while True:
            if self.camera_on is True:
                self.image_pour_widget = self.capture_image()
                self.image = cv2.resize(self.image_pour_widget,(int(256),int(256)))              
                self.Calcul()
                
                # Récupère les coordonnees XY
                coordonneesXY = self.output_host_mem[0][0][0][:][:]
                
                # Place les points
                width, height = (self.capture_width,self.capture_height)
                for point in coordonneesXY:
                    pt_Y = point[0]
                    pt_X = point[1]
                
                    # Calcule la position du point sur l'image
                    pt_image_X = pt_X * width
                    pt_image_Y = pt_Y * height

                    # Affiche le point sur l'image
                    self.image_pour_widget = cv2.circle(self.image_pour_widget, (int(pt_image_X),int(pt_image_Y)), radius=5, color=(0, 0, 255), thickness=-1)
                
                self.widget_image.value = bgr8_to_jpeg(self.image_pour_widget)
                time.sleep(0.001)

    # Définition du pipeline pour la caméra CSI
    def _gstreamer_pipeline_CSI(self):
        return("nvarguscamerasrc sensor-id=%d ! "
                "video/x-raw(memory:NVMM),"
                "width=(int)%d,height=(int)%d,"
                "format=(string)NV12, framerate=(fraction)%d/1 ! "
                "nvvidconv flip-method=%d ! "
                "video/x-raw,"
                "width=(int)%d,height=(int)%d,"
                "format=(string)BGRx ! videoconvert ! "
                "video/x-raw, format=(string)BGR ! "
                "appsink drop=true"
        %(self.capture_device,self.capture_width,self.capture_height,self.fps,self.flip, self.display_width,self.display_height))

    # Définition du pipeline pour la USB
    def _gstreamer_pipeline_USB(self):
        return("v4l2src device=/dev/video%d ! "
               "video/x-raw, width=(int)%d, height=(int)%d, framerate=(fraction)%d/1 ! "
               "videoflip method=%d ! "
               "videoconvert ! "
               "video/x-raw, format=(string)BGR ! appsink drop=true"
        %(self.capture_device,self.capture_width,self.capture_height,self.fps,self.flip))            

    # Routine pour arrêter le Thread
    def raise_exception(self):
        for id, thread in threading._active.items():
            if thread is self:
                thread_id = id
        res = ctypes.pythonapi.PyThreadState_SetAsyncExc(thread_id,ctypes.py_object(SystemExit))
        if res > 1:
            ctypes.pythonapi.PyThreadState_SetAsyncExc(thread_id, 0)
            print('Exception raise failure')

    def destroy(self):
        self.cudactx.detach()

### Interface de visualisation

In [6]:
import ipywidgets.widgets as widgets
import traitlets
from IPython.display import display
from jetbot import bgr8_to_jpeg

# Création de l'interface
image_widget = widgets.Image(format='jpeg', width=1024, height=768)

# Affichage de l'interface
display(widgets.VBox([image_widget]))

In [7]:
trt_inference_wrapper = TRTInference(repertoire_engine="model_jetson.engine",
                        widget_image=image_widget,
                        type_camera="USB",capture_device=1,
                        capture_width=1280,capture_height=960,
                        display_width=1280,display_height=960,
                        fps=30,flip=0)

Caméra initialisée
Chargement du moteur...
Allocation mémoire...


In [8]:
trt_inference_wrapper.start()
trt_inference_wrapper.camera_on = True

### Arrêt de la caméra

In [ ]:
trt_inference_wrapper.camera_on = False